In [2]:
import Pkg
Pkg.activate(".")
Pkg.add("CategoricalArrays")
Pkg.add("MAT")
Pkg.add("DataFrames")
Pkg.add("MLJ")
Pkg.add("MLJBase")
Pkg.instantiate()
Pkg.add("MLJLinearModels")
Pkg.instantiate()
Pkg.add("MLJTuning")
Pkg.instantiate()
Pkg.add("MLJModels")
Pkg.instantiate()
Pkg.add("Statistics")
Pkg.instantiate()
Pkg.add("StatsBase")
Pkg.instantiate()
Pkg.add("Random")
Pkg.instantiate()
# Define the packages
using Pkg
using CategoricalArrays
using MAT
using DataFrames
using MLJ
using LinearAlgebra
using Statistics
using Plots
using StatsPlots
using CSV

  Activating new project at `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM`
   Resolving package versions...
    Updating `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Project.toml`
  [324d7699] + CategoricalArrays v0.10.8
    Updating `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Manifest.toml`
  [324d7699] + CategoricalArrays v0.10.8
  [9a962f9c] + DataAPI v1.16.0
  [e1d29d7a] + Missings v1.2.0
  [ae029012] + Requires v1.3.1
  [10745b16] + Statistics v1.11.1
  [56f22d72] + Artifacts v1.11.0
  [9fa8497b] + Future v1.11.0
  [8f399da3] + Libdl v1.11.0
  [37e2e46d] + LinearAlgebra v1.11.0
  [de0858da] + Printf v1.11.0
  [9a3f8284] + Random v1.11.0
  [ea8e919c] + SHA v0.7.0
  [cf7118a7] + UUIDs v1.11.0
  [4ec0a83e] + Unicode v1.11.0
  [e66e0078] + CompilerSupportLibraries_jll v1.1.1+0
  [4536629a] + OpenBLAS_jll v0.3.27+1
  [8e850b90] + libblast

In [4]:
Pkg.add("ProgressMeter")
Pkg.instantiate()

   Resolving package versions...
    Updating `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Project.toml`
  [92933f4c] + ProgressMeter v1.10.4
  No Changes to `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Manifest.toml`


In [ ]:
using Random, Statistics, ProgressMeter

"""
    permutation_test_classifier(
        X::Tables.AbstractColumns, y::AbstractVector,
        model; cv=nothing, n_perm=1000, measure=accuracy, rng=Random.default_rng()
    ) → (p, observed, null_dist)

Exact-label permutation test for classification metrics.

* `model`     - an MLJ model instance (e.g. LogisticClassifier()).
* `cv`        - a pre-built `StratifiedCV` or `CV`; if `nothing` a 5-fold stratified CV is built.
* `n_perm`    - number of label shuffles.
* `measure`   - any MLJ measure (default = `accuracy`).

Returns the *p*-value, the observed score, and the vector of permuted scores.
"""
function permutation_test_classifier(X, y, model;
        cv=nothing, n_perm::Int=1000,
        measure=accuracy, rng=Random.default_rng())

    cv === nothing && (cv = StratifiedCV(nfolds=5, shuffle=true, rng=rng))
    obs_mach = machine(model, X, y)
    obs_result = evaluate!(obs_mach; resampling=cv, measures=[measure],
                           operation=predict_mode, verbosity=0)
    observed = mean(obs_result.per_fold[1])

    null_scores = Float64[]
    @showprogress 1 "Permutation test…" for _ in 1:n_perm
        y_perm = shuffle(rng, y)
        perm_mach = machine(model, X, y_perm)
        res = evaluate!(perm_mach; resampling=cv, measures=[measure],
                        operation=predict_mode, verbosity=0)
        push!(null_scores, mean(res.per_fold[1]))
    end
    # one-sided p-value (H_0: score is not better than chance)
    p = (count(>=(observed), null_scores) + 1) / (n_perm + 1)

    return p, observed, null_scores
end



permutation_test_regressor

In [52]:
# sub-MOA101, sub-MOA102, sub-MOA104, sub-MOA105, sub-MOA107, sub-MOA108, sub-MOA109, sub-MOA110, sub-MOA111, sub-MOA112, sub-MOA114,
# sub-MOA115, sub-MOA116, sub-MOA118, sub-MOA121, sub-MOA122, sub-MOA123, sub-MOA124, sub-MOA126, sub-MOA127, sub-MOA128, sub-MOA130,
# sub-MOA131, sub-MOA133, sub-MOA134, sub-MOA135

# Binary treatment response categories
y_binary = ("Response","Response","Response","No Response","Response","No Response","Response","No Response","Response","Response",
    "No Response","No Response","Response","Response","No Response","No Response","No Response","No Response","Response","Response",
    "No Response", "No Response" ,"No Response", "No Response","Response", "No Response")

# Tri label response categories
y_tri = ("Partial Response","Partial Response","Remission","No Response","Partial Response","No Response","Partial Response","No Response",
    "Remission", "Remission","Partial Response","Partial Response","Partial Response","Remission","No Response","No Response",
    "Partial Response","No Response","Remission","Partial Response","Partial Response","Partial Response","No Response","No Response",
    "Remission","No Response")

# Quad label response categories
y_quad = ("Partial Response","Partial Response","Remission","Stable","Partial Response","Deterioration","Partial Response","Stable",
    "Remission","Remission","Partial Response","Partial Response","Partial Response","Remission","Stable","Deterioration", "Stable",
    "Stable","Remission","Partial Response","Partial Response","Partial Response","Stable","Stable","Remission","Stable")

# Penta label response categories
y_penta = ("Strong Response","Strong Response","Remission","Stable","Strong Response","Deterioration","Strong Response","Stable",
    "Remission","Remission","Mild Response","Mild Response","Strong Response","Remission","Stable","Deterioration","Stable","Stable",
    "Remission","Strong Response","Mild Response","Mild Response","Stable","Stable","Remission","Stable")



# Convert to CategoricalArray
y_binary = categorical(collect(y_binary))
y_tri = categorical(collect(y_tri))
y_quad = categorical(collect(y_quad))
y_penta = categorical(collect(y_penta))


26-element CategoricalArray{String,1,UInt32}:
 "Strong Response"
 "Strong Response"
 "Remission"
 "Stable"
 "Strong Response"
 "Deterioration"
 "Strong Response"
 "Stable"
 "Remission"
 "Remission"
 ⋮
 "Stable"
 "Remission"
 "Strong Response"
 "Mild Response"
 "Mild Response"
 "Stable"
 "Stable"
 "Remission"
 "Stable"

In [ ]:
# classification – permutation test outcomes
LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
p, acc, null_acc = permutation_test_classifier(X_df, y_binary,
                                               LogisticClassifier(); n_perm=500)
println("Permutation p-value (accuracy) = $p")


p_tri, acc_tri, null_acc_tri = permutation_test_classifier(X_df, y_tri,
                                                           LogisticClassifier(); n_perm=500)

println("Permutation p-value (tri accuracy) = $p_tri")

p_quad, acc_quad, null_acc_quad = permutation_test_classifier(X_df, y_quad,
                                                               LogisticClassifier(); n_perm=500)
println("Permutation p-value (quad accuracy) = $p_quad")

p_penta, acc_penta, null_acc_penta = permutation_test_classifier(X_df, y_penta,
                                                                   LogisticClassifier(); n_perm=500)
println("Permutation p-value (penta accuracy) = $p_penta")


import MLJLinearModels ✔
Permutation p-value (accuracy) = 0.05189620758483034

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159



Permutation p-value (tri accuracy) = 0.05788423153692615
Permutation p-value (quad accuracy) = 0.7265469061876247
Permutation p-value (penta accuracy) = 0.281437125748503


In [54]:
# List of subjects that have some degree of depression
target_subjects = [
    "sub-MOA101", "sub-MOA102", "sub-MOA104", "sub-MOA105", "sub-MOA107", "sub-MOA108", "sub-MOA109", "sub-MOA110", "sub-MOA111",
    "sub-MOA112", "sub-MOA114", "sub-MOA115", "sub-MOA116", "sub-MOA118", "sub-MOA121", "sub-MOA122", "sub-MOA123","sub-MOA124", 
    "sub-MOA126", "sub-MOA127", "sub-MOA128", "sub-MOA130", "sub-MOA131", "sub-MOA133", "sub-MOA134", "sub-MOA135"]

# Base path to your subject folders
base_path = "Spectral_DCM_Collection_Diag_4x4"

# Collect valid file paths
valid_files = String[]

for subj in target_subjects
    subj_path = joinpath(base_path, subj)
    ses_path = joinpath(subj_path, "ses-b0")
    glm_path = joinpath(ses_path, "glm")
    dcm_file = joinpath(glm_path, "spDCM_DMN.mat")

    if !isdir(ses_path)
        @warn "Missing session folder: $ses_path"
    elseif !isfile(dcm_file)
        @warn "Missing spDCM_DMN.mat for $subj"
    else
        push!(valid_files, dcm_file)
    end
end

println("✅ Found Spectral_DCM.mat for $(length(valid_files)) out of $(length(target_subjects)) subjects.")


✅ Found Spectral_DCM.mat for 26 out of 26 subjects.


In [56]:
# Extract A matrix features as a flat 16-element vector
function extract_features(file)
    mat = matread(file)
    A = mat["params"]  # 4×4 matrix
    return vec(Matrix(A))  # Flatten to 16-element vector
end

extract_features (generic function with 1 method)

In [57]:
# Create feature dataset
X = hcat([extract_features(file) for file in valid_files]...)'

X_df = DataFrame(X, :auto)  # convert to MLJ-compatible table

# Ensure that the number of subjects match in X_df and Y labels of different modalities
@assert size(X_df, 1) == length(y_binary) "Mismatch between number of samples in X and y_binary"
@assert size(X_df, 1) == length(y_tri) "Mismatch between number of samples in X and y_tri"
@assert size(X_df, 1) == length(y_quad) "Mismatch between number of samples in X and y_quad"
@assert size(X_df, 1) == length(y_penta) "Mismatch between number of samples in X and y_penta"

In [58]:
function evaluate_logistic_model(X_df, y, nfolds::Int, label_type::String)
    # Load model
    LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
    pipe = LogisticClassifier()
    mach = machine(pipe, X_df, y)

    # Setup evaluation
    measures = [accuracy, MulticlassFScore(), ConfusionMatrix()]
    cv = StratifiedCV(nfolds=nfolds, shuffle=true, rng=42)

    # Evaluate
    eval_result = evaluate!(
        mach,
        resampling=cv,
        measures=measures,
        operation=predict_mode,
        verbosity=0,
        check_measure=false
    )

    accs = eval_result.per_fold[1]
    f1s = eval_result.per_fold[2]
    cmats = eval_result.per_fold[3]

    macro_precisions = Float64[]
    macro_recalls = Float64[]

    for cm in cmats
        cmatrix = cm.mat
        TP = diag(cmatrix)
        FP = sum(cmatrix, dims=1)' .- TP
        FN = sum(cmatrix, dims=2) .- TP
        support = sum(cmatrix, dims=2)[:]
        total = sum(support)

        class_precisions = map((tp, fp) -> (tp + fp == 0 ? NaN : tp / (tp + fp)), TP, FP)
        class_recalls    = map((tp, fn) -> (tp + fn == 0 ? NaN : tp / (tp + fn)), TP, FN)

        macro_precision = mean(skipmissing(class_precisions))
        macro_recall    = mean(skipmissing(class_recalls))

        push!(macro_precisions, macro_precision)
        push!(macro_recalls, macro_recall)
    end

    # Collect metrics in DataFrame
    metrics_df = DataFrame(
        Fold = 1:nfolds,
        Accuracy = accs,
        F1_Score = f1s,
        Macro_Precision = macro_precisions,
        Macro_Recall = macro_recalls
    )

    # Add row for averages
    avg_row = DataFrame(
        Fold = ["Average"],
        Accuracy = [mean(skipmissing(accs))],
        F1_Score = [mean(skipmissing(f1s))],
        Macro_Precision = [mean(skipmissing(macro_precisions))],
        Macro_Recall = [mean(skipmissing(macro_recalls))]
    )

    metrics_table = vcat(metrics_df, avg_row)

    # Save table as CSV instead (more robust and universally usable)
    csv_table_path = "tables/table_$(nfolds)_grouped_metrics_logistics_$(label_type).csv"
    CSV.write(csv_table_path, metrics_table)

    # Prepare DataFrame for plotting
    metrics = ["Accuracy", "F1 Score", "Macro Precision", "Macro Recall"]
    metric_values = [accs, f1s, macro_precisions, macro_recalls]

    plot_df = DataFrame(
        Fold = repeat(1:nfolds, outer=length(metrics)),
        Metric = repeat(metrics, inner=nfolds),
        Value = vcat(metric_values...)
    )

    # Plot grouped bars
    @df plot_df groupedbar(
        string.(:Fold), :Value, group=:Metric,
        bar_position=:dodge,
        bar_width=0.2,
        xlabel="Fold", ylabel="Metric Value",
        yticks=0:0.05:1.0,
        title="Logistic Classification Fold-wise Metrics",
        legend=:outertop,
        size=(750, 500),
        guidefontsize=10,
        tickfontsize=10,
        dpi=300
    )

    savefig("figures/$(nfolds)_grouped_metrics_logistics_$(label_type).png")
end

evaluate_logistic_model (generic function with 1 method)

In [80]:
# #Evaluate logistic model classifier for 3 fold cross validation
# evaluate_logistic_model(X_df, y_binary, 3, "binary")
# evaluate_logistic_model(X_df, y_tri, 3, "tri")
# evaluate_logistic_model(X_df, y_quad, 3, "quad")
# evaluate_logistic_model(X_df, y_penta, 3, "penta")


# #Evaluate logistic model classifier for 4 fold cross validation
# evaluate_logistic_model(X_df, y_binary, 4, "binary")
# evaluate_logistic_model(X_df, y_tri, 4, "tri")
# evaluate_logistic_model(X_df, y_quad, 4, "quad")
# evaluate_logistic_model(X_df, y_penta, 4, "penta")


#Evaluate logistic model classifier for 5 fold cross validation
evaluate_logistic_model(X_df, y_binary, 5, "binary")
evaluate_logistic_model(X_df, y_tri, 5, "tri")
evaluate_logistic_model(X_df, y_quad, 5, "quad")
evaluate_logistic_model(X_df, y_penta, 5, "penta")


import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔


"/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/5_grouped_metrics_logistics_penta.png"

In [81]:
LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
p, acc, null_acc = permutation_test_classifier(X_df, y_binary,
                                               LogisticClassifier(); n_perm=500)
println("Permutation p-value (accuracy) = $p")


p_tri, acc_tri, null_acc_tri = permutation_test_classifier(X_df, y_tri,
                                                           LogisticClassifier(); n_perm=500)

println("Permutation p-value (tri accuracy) = $p_tri")

p_quad, acc_quad, null_acc_quad = permutation_test_classifier(X_df, y_quad,
                                                               LogisticClassifier(); n_perm=500)
println("Permutation p-value (quad accuracy) = $p_quad")

p_penta, acc_penta, null_acc_penta = permutation_test_classifier(X_df, y_penta,
                                                                   LogisticClassifier(); n_perm=500)
println("Permutation p-value (penta accuracy) = $p_penta")




import MLJLinearModels ✔
Permutation p-value (accuracy) = 0.03992015968063872

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159



Permutation p-value (tri accuracy) = 0.11976047904191617
Permutation p-value (quad accuracy) = 0.36726546906187624
Permutation p-value (penta accuracy) = 0.2754491017964072


In [63]:
function evaluate_multinomial_classifier_model(X_df, y, nfolds::Int, label_type::String)
    # Load model
    MultinomialClassifier = @load MultinomialClassifier pkg=MLJLinearModels
    pipe = MultinomialClassifier()
    mach = machine(pipe, X_df, y)

    # Setup evaluation
    measures = [accuracy, MulticlassFScore(), ConfusionMatrix()]
    cv = StratifiedCV(nfolds=nfolds, shuffle=true, rng=42)

    # Evaluate
    eval_result = evaluate!(
        mach,
        resampling=cv,
        measures=measures,
        operation=predict_mode,
        verbosity=0,
        check_measure=false
    )

    accs = eval_result.per_fold[1]
    f1s = eval_result.per_fold[2]
    cmats = eval_result.per_fold[3]

    macro_precisions = Float64[]
    macro_recalls = Float64[]

    for cm in cmats
        cmatrix = cm.mat
        TP = diag(cmatrix)
        FP = sum(cmatrix, dims=1)' .- TP
        FN = sum(cmatrix, dims=2) .- TP
        support = sum(cmatrix, dims=2)[:]
        total = sum(support)

        class_precisions = map((tp, fp) -> (tp + fp == 0 ? NaN : tp / (tp + fp)), TP, FP)
        class_recalls    = map((tp, fn) -> (tp + fn == 0 ? NaN : tp / (tp + fn)), TP, FN)

        macro_precision = mean(skipmissing(class_precisions))
        macro_recall    = mean(skipmissing(class_recalls))

        push!(macro_precisions, macro_precision)
        push!(macro_recalls, macro_recall)
    end

    # Collect metrics in DataFrame
    metrics_df = DataFrame(
        Fold = 1:nfolds,
        Accuracy = accs,
        F1_Score = f1s,
        Macro_Precision = macro_precisions,
        Macro_Recall = macro_recalls
    )

    # Add row for averages
    avg_row = DataFrame(
        Fold = ["Average"],
        Accuracy = [mean(skipmissing(accs))],
        F1_Score = [mean(skipmissing(f1s))],
        Macro_Precision = [mean(skipmissing(macro_precisions))],
        Macro_Recall = [mean(skipmissing(macro_recalls))]
    )

    metrics_table = vcat(metrics_df, avg_row)

    # Save table as CSV instead (more robust and universally usable)
    csv_table_path = "tables/table_$(nfolds)_grouped_metrics_multinomial_$(label_type).csv"
    CSV.write(csv_table_path, metrics_table)

    # Prepare DataFrame for plotting
    metrics = ["Accuracy", "F1 Score", "Macro Precision", "Macro Recall"]
    metric_values = [accs, f1s, macro_precisions, macro_recalls]

    plot_df = DataFrame(
        Fold = repeat(1:nfolds, outer=length(metrics)),
        Metric = repeat(metrics, inner=nfolds),
        Value = vcat(metric_values...)
    )

    # Plot grouped bars
    @df plot_df groupedbar(
        string.(:Fold), :Value, group=:Metric,
        bar_position=:dodge,
        bar_width=0.2,
        xlabel="Fold", ylabel="Metric Value",
        yticks=0:0.05:1.0,
        title="Multinomial Classification Fold-wise Metrics",
        legend=:outertop,
        size=(750, 500),
        guidefontsize=10,
        tickfontsize=10,
        dpi=300
    )

    savefig("figures/$(nfolds)_grouped_metrics_multinomial_$(label_type).png")
end

evaluate_multinomial_classifier_model (generic function with 1 method)

In [64]:
#Evaluate multinomial model classifier for 3 fold cross validation
evaluate_multinomial_classifier_model(X_df, y_binary, 3, "binary")
evaluate_multinomial_classifier_model(X_df, y_tri, 3, "tri")
evaluate_multinomial_classifier_model(X_df, y_quad, 3, "quad")
evaluate_multinomial_classifier_model(X_df, y_penta, 3, "penta")

#Evaluate multinomial model classifier for 4 fold cross validation
evaluate_multinomial_classifier_model(X_df, y_binary, 4, "binary")
evaluate_multinomial_classifier_model(X_df, y_tri, 4, "tri")
evaluate_multinomial_classifier_model(X_df, y_quad, 4, "quad")
evaluate_multinomial_classifier_model(X_df, y_penta, 4, "penta")

#Evaluate multinomial model classifier for 5 fold cross validation
evaluate_multinomial_classifier_model(X_df, y_binary, 5, "binary")
evaluate_multinomial_classifier_model(X_df, y_tri, 5, "tri")
evaluate_multinomial_classifier_model(X_df, y_quad, 5, "quad")
evaluate_multinomial_classifier_model(X_df, y_penta, 5, "penta")

import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔


...

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔


"/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/5_grouped_metrics_multinomial_penta.png"

In [65]:
LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
p, acc, null_acc = permutation_test_classifier(X_df, y_binary,
                                               LogisticClassifier(); n_perm=500)
println("Permutation p-value (accuracy) = $p")


p_tri, acc_tri, null_acc_tri = permutation_test_classifier(X_df, y_tri,
                                                           LogisticClassifier(); n_perm=500)

println("Permutation p-value (tri accuracy) = $p_tri")

p_quad, acc_quad, null_acc_quad = permutation_test_classifier(X_df, y_quad,
                                                               LogisticClassifier(); n_perm=500)
println("Permutation p-value (quad accuracy) = $p_quad")

p_penta, acc_penta, null_acc_penta = permutation_test_classifier(X_df, y_penta,
                                                                   LogisticClassifier(); n_perm=500)
println("Permutation p-value (penta accuracy) = $p_penta")

import MLJLinearModels ✔
Permutation p-value (accuracy) = 0.08582834331337326

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159



Permutation p-value (tri accuracy) = 0.25149700598802394
Permutation p-value (quad accuracy) = 0.33532934131736525
Permutation p-value (penta accuracy) = 0.1277445109780439


In [66]:
Pkg.add("MLJScikitLearnInterface")
Pkg.instantiate()

   Resolving package versions...
  No Changes to `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Project.toml`
  No Changes to `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Manifest.toml`


In [67]:
using MLJScikitLearnInterface
function evaluate_random_forest_model(X_df, y, nfolds::Int, label_type::String)
    # Load model
    RandomForestClassifier = @load RandomForestClassifier pkg=MLJScikitLearnInterface
    pipe = RandomForestClassifier()
    mach = machine(pipe, X_df, y)

    # Setup evaluation
    measures = [accuracy, MulticlassFScore(), ConfusionMatrix()]
    cv = StratifiedCV(nfolds=nfolds, shuffle=true, rng=42)

    # Evaluate
    eval_result = evaluate!(
        mach,
        resampling=cv,
        measures=measures,
        operation=predict_mode,
        verbosity=0,
        check_measure=false
    )

    accs = eval_result.per_fold[1]
    f1s = eval_result.per_fold[2]
    cmats = eval_result.per_fold[3]

    macro_precisions = Float64[]
    macro_recalls = Float64[]

    for cm in cmats
        cmatrix = cm.mat
        TP = diag(cmatrix)
        FP = sum(cmatrix, dims=1)' .- TP
        FN = sum(cmatrix, dims=2) .- TP
        support = sum(cmatrix, dims=2)[:]
        total = sum(support)

        class_precisions = map((tp, fp) -> (tp + fp == 0 ? NaN : tp / (tp + fp)), TP, FP)
        class_recalls    = map((tp, fn) -> (tp + fn == 0 ? NaN : tp / (tp + fn)), TP, FN)

        macro_precision = mean(skipmissing(class_precisions))
        macro_recall    = mean(skipmissing(class_recalls))

        push!(macro_precisions, macro_precision)
        push!(macro_recalls, macro_recall)
    end

    # Collect metrics in DataFrame
    metrics_df = DataFrame(
        Fold = 1:nfolds,
        Accuracy = accs,
        F1_Score = f1s,
        Macro_Precision = macro_precisions,
        Macro_Recall = macro_recalls
    )

    # Add row for averages
    avg_row = DataFrame(
        Fold = ["Average"],
        Accuracy = [mean(skipmissing(accs))],
        F1_Score = [mean(skipmissing(f1s))],
        Macro_Precision = [mean(skipmissing(macro_precisions))],
        Macro_Recall = [mean(skipmissing(macro_recalls))]
    )

    metrics_table = vcat(metrics_df, avg_row)

    # Save table as CSV instead (more robust and universally usable)
    csv_table_path = "tables/table_$(nfolds)_grouped_metrics_random_forest_$(label_type).csv"
    CSV.write(csv_table_path, metrics_table)

    # Prepare DataFrame for plotting
    metrics = ["Accuracy", "F1 Score", "Macro Precision", "Macro Recall"]
    metric_values = [accs, f1s, macro_precisions, macro_recalls]

    plot_df = DataFrame(
        Fold = repeat(1:nfolds, outer=length(metrics)),
        Metric = repeat(metrics, inner=nfolds),
        Value = vcat(metric_values...)
    )

    # Plot grouped bars
    @df plot_df groupedbar(
        string.(:Fold), :Value, group=:Metric,
        bar_position=:dodge,
        bar_width=0.2,
        xlabel="Fold", ylabel="Metric Value",
        yticks=0:0.05:1.0,
        title="Random Forest Classification Fold-wise Metrics",
        legend=:outertop,
        size=(750, 500),
        guidefontsize=10,
        tickfontsize=10,
        dpi=300
    )

    savefig("figures/$(nfolds)_grouped_metrics_random_forest_$(label_type).png")
end

evaluate_random_forest_model (generic function with 1 method)

In [68]:
#Evaluate random forest model classifier for 3 fold cross validation
evaluate_random_forest_model(X_df, y_binary, 3, "binary")
evaluate_random_forest_model(X_df, y_tri, 3, "tri")
evaluate_random_forest_model(X_df, y_quad, 3, "quad")
evaluate_random_forest_model(X_df, y_penta, 3, "penta")

#Evaluate random forest model classifier for 4 fold cross validation
evaluate_random_forest_model(X_df, y_binary, 4, "binary")
evaluate_random_forest_model(X_df, y_tri, 4, "tri")
evaluate_random_forest_model(X_df, y_quad, 4, "quad")
evaluate_random_forest_model(X_df, y_penta, 4, "penta")

#Evaluate random forest model classifier for 5 fold cross validation
evaluate_random_forest_model(X_df, y_binary, 5, "binary")
evaluate_random_forest_model(X_df, y_tri, 5, "tri")
evaluate_random_forest_model(X_df, y_quad, 5, "quad")
evaluate_random_forest_model(X_df, y_penta, 5, "penta")

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


...

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJScikitLearnInterface ✔


"/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/5_grouped_metrics_random_forest_penta.png"

In [69]:
LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
p, acc, null_acc = permutation_test_classifier(X_df, y_binary,
                                               LogisticClassifier(); n_perm=500)
println("Permutation p-value (accuracy) = $p")


p_tri, acc_tri, null_acc_tri = permutation_test_classifier(X_df, y_tri,
                                                           LogisticClassifier(); n_perm=500)

println("Permutation p-value (tri accuracy) = $p_tri")

p_quad, acc_quad, null_acc_quad = permutation_test_classifier(X_df, y_quad,
                                                               LogisticClassifier(); n_perm=500)
println("Permutation p-value (quad accuracy) = $p_quad")

p_penta, acc_penta, null_acc_penta = permutation_test_classifier(X_df, y_penta,
                                                                   LogisticClassifier(); n_perm=500)
println("Permutation p-value (penta accuracy) = $p_penta")

import MLJLinearModels ✔
Permutation p-value (accuracy) = 0.6766467065868264

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159



Permutation p-value (tri accuracy) = 0.09780439121756487
Permutation p-value (quad accuracy) = 0.4411177644710579
Permutation p-value (penta accuracy) = 0.2554890219560878
